ISLRパッケージを読み込む

In [2]:
library(ISLR)

Warning message:
"package 'ISLR' was built under R version 4.0.2"


fixでHittersデータを出力

In [4]:
fix(Hitters)

na.omitで欠損値が存在するデータの行を削除

In [5]:
Hitters=na.omit(Hitters)

取り除いた後にis.naで欠損値の数を確認する

In [6]:
sum(is.na(Hitters))

[1] 0

x行列とyベクトルを渡す

In [8]:
x=model.matrix(Salary~.,Hitters)[,-1]

In [10]:
y=Hitters$Salary

リッジ回帰をやる
alpha=0ならばリッジ回帰,alpha=1ならばlasso回帰になる

In [11]:
library(glmnet)

Warning message:
"package 'glmnet' was built under R version 4.0.2"
Loading required package: Matrix

Loaded glmnet 4.0-2



In [12]:
grid=10^seq(10,-2,length=100)

In [13]:
ridge.mod=glmnet(x,y,alpha=0,lambda=grid)

リッジ回帰係数ベクトルを行列形式で表示する
λ=100なので20×100行列となる

In [14]:
dim(coef(ridge.mod))

[1]  20 100

λ=11,498の時の係数と$ℓ_2$ノルム

In [15]:
ridge.mod$lambda[50]

[1] 11497.57

In [16]:
coef(ridge.mod)[,50]

(Intercept)         AtBat          Hits         HmRun          Runs 
407.356050200   0.036957182   0.138180344   0.524629976   0.230701523 
          RBI         Walks         Years        CAtBat         CHits 
  0.239841459   0.289618741   1.107702929   0.003131815   0.011653637 
       CHmRun         CRuns          CRBI        CWalks       LeagueN 
  0.087545670   0.023379882   0.024138320   0.025015421   0.085028114 
    DivisionW       PutOuts       Assists        Errors    NewLeagueN 
 -6.215440973   0.016482577   0.002612988  -0.020502690   0.301433531

λ=705の時の係数と$ℓ_2$ノルム

In [17]:
ridge.mod$lambda[60]

[1] 705.4802

In [18]:
coef(ridge.mod)[,60]

(Intercept)        AtBat         Hits        HmRun         Runs          RBI 
 54.32519950   0.11211115   0.65622409   1.17980910   0.93769713   0.84718546 
       Walks        Years       CAtBat        CHits       CHmRun        CRuns 
  1.31987948   2.59640425   0.01083413   0.04674557   0.33777318   0.09355528 
        CRBI       CWalks      LeagueN    DivisionW      PutOuts      Assists 
  0.09780402   0.07189612  13.68370191 -54.65877750   0.11852289   0.01606037 
      Errors   NewLeagueN 
 -0.70358655   8.61181213

predict()でλ=50としたときのリッジ回帰係数を求める

In [19]:
predict(ridge.mod,s=50,type="coefficients")[1:20,]

(Intercept)         AtBat          Hits         HmRun          Runs 
 4.876610e+01 -3.580999e-01  1.969359e+00 -1.278248e+00  1.145892e+00 
          RBI         Walks         Years        CAtBat         CHits 
 8.038292e-01  2.716186e+00 -6.218319e+00  5.447837e-03  1.064895e-01 
       CHmRun         CRuns          CRBI        CWalks       LeagueN 
 6.244860e-01  2.214985e-01  2.186914e-01 -1.500245e-01  4.592589e+01 
    DivisionW       PutOuts       Assists        Errors    NewLeagueN 
-1.182011e+02  2.502322e-01  1.215665e-01 -3.278600e+00 -9.496680e+00

リッジ回帰とlassoのテスト誤差を推定する
無造作に1からnまでの内の部分集合をとる

乱数のseedを設定

In [20]:
set.seed(1)
train=sample(1:nrow(x),nrow(x)/2)
test=(-train)
y.test=y[test]

訓練データにリッジ回帰を当てはめ,λ=4としてテストMSEを評価する

In [22]:
ridge.mod=glmnet(x[train,],y[train],alpha=0,lambda=grid,thresh=1e-12)

In [23]:
ridge.pred=predict(ridge.mod,s=4,newx=x[test,])

In [24]:
mean((ridge.pred-y.test)^2)

[1] 142199.2

上記がテストMSEの値だが,もし切片のみのモデルを当てはめたならば,テストデータの予測に訓練データの平均を使ったはずである
個の場合のテストMSEは以下式で計算できる

In [25]:
mean((mean(y[train])-y.test)^2)

[1] 224669.9